# 

Домашняя работа

Будем работать с датасетом подержанных машин https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes.

Задача - построить прогноз цены машины по ее данным (год выпуска, пробег, расход, размер двигателя, тип коробки передач и тд). 

In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_digits
from sklearn.metrics import balanced_accuracy_score, accuracy_score, confusion_matrix, classification_report, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from matplotlib.colors import ListedColormap
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

## Easy

Построить прогноз по одной марке машин при помощи решающих деревьев, взять только числовые признаки. В качестве метрики взять `r2_score`. Оценить важность признаков, проинтерпретировать модель.

In [2]:
data = pd.read_csv('../data/ford.csv')
data = data.drop(columns=['model', 'transmission', 'fuelType'])
y = data['price']
x = data.drop(columns=['price'])

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0)
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [4]:
pred = model.predict(x_test)
accurasy = r2_score(pred, y_test)
print(accurasy)

0.8270654416882601


In [5]:
model.feature_importances_

array([0.02807221, 0.78489126, 0.07445256, 0.0993855 , 0.01319848])

In [6]:
x.head()

,year,mileage,tax,mpg,engineSize
0,2017,15944,150,57.7,1.0
1,2018,9083,150,57.7,1.0
2,2017,12456,150,57.7,1.0
3,2019,10460,145,40.3,1.5
4,2019,1482,145,48.7,1.0


Единственный важный критерий - mileage, остальные не имеют значения.

## Normal

Объединить в один датафрейм данные по всем маркам машин. Преобразовать категориальные признаки.

Построить еще несколько моделей, используя подбор гиперпараметров. Сравнить между собой все построенные модели.

In [7]:
all_data = pd.concat([pd.read_csv('../data/vw.csv'), pd.read_csv('../data/vauxhall.csv'),
                     pd.read_csv('../data/toyota.csv'), pd.read_csv('../data/skoda.csv'), 
                     pd.read_csv('../data/merc.csv'), pd.read_csv('../data/hyundi.csv'),
                     pd.read_csv('../data/ford.csv'), pd.read_csv('../data/focus.csv'),
                     pd.read_csv('../data/cclass.csv'), pd.read_csv('../data/bmw.csv'), pd.read_csv('../data/audi.csv')], ignore_index=True)
all_data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
0,T-Roc,2019,25000,Automatic,13904,Diesel,145.0,49.6,2.0,NaN
1,T-Roc,2019,26883,Automatic,4562,Diesel,145.0,49.6,2.0,NaN
2,T-Roc,2019,20000,Manual,7414,Diesel,145.0,50.4,2.0,NaN
3,T-Roc,2019,33492,Automatic,4825,Petrol,145.0,32.5,2.0,NaN
4,T-Roc,2019,22900,Semi-Auto,6500,Petrol,150.0,39.8,1.5,NaN


In [8]:
all_data['model'] = LabelEncoder().fit_transform(all_data['model'])
all_data['transmission'] = LabelEncoder().fit_transform(all_data['transmission'])
all_data['fuelType'] = LabelEncoder().fit_transform(all_data['fuelType'])

In [9]:
all_data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
0,153,2019,25000,0,13904,0,145.0,49.6,2.0,NaN
1,153,2019,26883,0,4562,0,145.0,49.6,2.0,NaN
2,153,2019,20000,1,7414,0,145.0,50.4,2.0,NaN
3,153,2019,33492,0,4825,4,145.0,32.5,2.0,NaN
4,153,2019,22900,3,6500,4,150.0,39.8,1.5,NaN


In [10]:
sampled_data = data.sample(n=10000, random_state=0)
y = sampled_data['price']
x = sampled_data.drop(columns=['price'])

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0)

In [11]:
model_tree = GridSearchCV(DecisionTreeClassifier(), {'max_depth': [None, 5, 10],
                 'min_samples_split': [2, 5, 10]}, cv=5)
model_tree.fit(x_train, y_train)
model_tree.best_estimator_

pred_tree = model_tree.predict(x_test)
accurasy_tree = r2_score(pred_tree, y_test)


/home/iulia/miniconda3/envs/my-environment/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [12]:
sampled_data = data.sample(n=2000, random_state=0)
y = sampled_data['price']
x = sampled_data.drop(columns=['price'])

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0)

In [13]:
model_forest = GridSearchCV(RandomForestClassifier(),{'n_estimators': [100, 200, 300],
                 'max_depth': [None, 10, 20],
                 'min_samples_split': [2, 5, 10]}, cv=5)
model_forest.fit(x_train, y_train)
model_forest.best_estimator_

pred_forest = model_forest.predict(x_test)
accurasy_forest = r2_score(pred_forest, y_test)

/home/iulia/miniconda3/envs/my-environment/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [14]:
print(accurasy_tree)
print(accurasy_forest)

0.7377476589221963
0.719475417978596


Получили, что DecisionTreeClassifier() отработал лучше, но незначительно.